In [97]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests 
import math 

In [98]:
# Read the stock tickers
IEX_CLOUD_API_TOKEN = 'Val'
stock_tickers = pd.read_csv('/Users/davidpekar/PycharmProjects/AlgorithmicTrading/venv/lib/python3.8/sp_500_stocks.csv')

In [99]:
# Initialize the pandas dataframe
my_columns2 = ["Ticker", "Price", "MarketCap", "Number of Shares to Buy"]
data_frame = pd.DataFrame(columns=my_columns)
#api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}'
#    data = requests.get(api_url).json()

In [100]:
# execute batch api calls to make api calls more efficient
# split the array into 100 elements
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [138]:
# hold the variables in a list
symbol_groups = list(chunks(stock_tickers['Ticker'], 100))
# create a list of strings that will be passed into the api url
symbol_strings = []
for x in range(len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[x]))
print(symbol_strings)

['A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA', 'CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD', 'GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX

In [139]:
# create data_frame
# batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
my_columns = ["Ticker", "Stock Price", "MarketCap", "Number Of Shares to Buy"]
final_data_frame = pd.DataFrame(columns=my_columns)
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        if symbol == 'WLTW':
            continue
        final_data_frame = final_data_frame.append(
            pd.Series([symbol,
                       data[symbol]['quote']['latestPrice'],
                       data[symbol]['quote']['marketCap'],
                       'N/A'],
                      index=my_columns),
            ignore_index=True)
final_data_frame = final_data_frame[:501]
final_data_frame

,Ticker,Stock Price,MarketCap,Number Of Shares to Buy
0,A,148.12,42804067548,N/A
1,AAL,18.76,12513946687,N/A
2,AAP,231.52,14027131122,N/A
3,AAPL,175.52,2925605433213,N/A
4,ABBV,147.75,262146549461,N/A
...,...,...,...,...
496,XRAY,55.05,12037578879,N/A
497,XRX,22.40,3939061500,N/A
498,XYL,94.07,16704820858,N/A
499,YUM,127.44,37290059416,N/A


In [140]:
# calculate the number of shares to buy
portfolio_amount = int(input("Enter a portfolio amount "))
amount_per_stock = math.floor(portfolio_amount/len(final_data_frame.index))

for i in range(0,len(final_data_frame.index)):
     final_data_frame.loc[i, 'Number Of Shares to Buy'] = math.floor(amount_per_stock / final_data_frame['Stock Price'][i])
    
final_data_frame            

Enter a portfolio amount 10000000


,Ticker,Stock Price,MarketCap,Number Of Shares to Buy
0,A,148.12,42804067548,134
1,AAL,18.76,12513946687,1063
2,AAP,231.52,14027131122,86
3,AAPL,175.52,2925605433213,113
4,ABBV,147.75,262146549461,135
...,...,...,...,...
496,XRAY,55.05,12037578879,362
497,XRX,22.40,3939061500,891
498,XYL,94.07,16704820858,212
499,YUM,127.44,37290059416,156
